### Junk script so i can learn this problem, ignore mostly

In [84]:
import numpy as np
import numpy.linalg as LA
from scipy.linalg import block_diag, solve_triangular

Dimensions
* $n$ - number of training examples
* $d$ - feature dimension
* $r$ - rank of $X$
* $P$ - total enumerations of hidden neurons / ReLU activations
* $P_S$ - number of hidden neurons / samples of $D_i$ matrices

Matrices
* $X \in \mathbb{R}^{n \times d}$ - training data matrix
* $y \in \mathbb{R}^{n}$ - training labels

* $D_i \in \mathbb{R}^{n \times n}, \ i = 1,\dots,P_S$ - diagonal matrices sampling hidden layer activations
* $F_i = D_i X \in \mathbb{R}^{n \times d}, \ i = 1,\dots,P_S$
* $G_i = (2 D_i - I_n) X \in \mathbb{R}^{n \times d}, \ i = 1,\dots,P_S$
 
Primal Variables  
* $v_i \in \mathbb{R}^{d}, \ i = 1,\dots,P_S$ - vector weights 1
* $w_i \in \mathbb{R}^{d}, \ i = 1,\dots,P_S$ - vector weights 2

Introduced Variables
* $u_i \in \mathbb{R}^{d}, \ i = 1,\dots,P_S; \quad \quad u_i = v_i \ \forall i$ 
* $z_i \in \mathbb{R}^{d}, \ i = 1,\dots,P_S; \quad \quad z_i = w_i \ \forall i$ 

Slack Variables
* $s_i = G_i v_i \in \mathbb{R}^{n}, \ i = 1,\dots,P_S$
* $t_i = G_i w_i \in \mathbb{R}^{n}, \ i = 1,\dots,P_S$

Composite Vectors and Matrices
* $u := \begin{bmatrix} u_1^T & \dots & u_{P_S}^T & z_1^T & \dots & z_{P_S}^T \end{bmatrix}^T \in \mathbb{R}^{2 d P_s}$

* $v := \begin{bmatrix} v_1^T & \dots & v_{P_S}^T & w_1^T & \dots & w_{P_S}^T \end{bmatrix}^T \in \mathbb{R}^{2 d P_s}$
* $s := \begin{bmatrix} s_1^T & \dots & s_{P_S}^T & t_1^T & \dots & t_{P_S}^T \end{bmatrix}^T \in \mathbb{R}^{2 n P_s}$
* $F = \begin{bmatrix} F_1 & \dots & F_{P_S} & -F_1 & \dots & -F_{P_S} \end{bmatrix} \in \mathbb{R}^{n \times (2 d P_S)}, \ i = 1,\dots,P_S$
* $G = \text{blockdiag}(G_1, \ \dots, \ G_{P_S}, \ G_1, \ \dots, \ G_{P_S}) \in \mathbb{R}^{n \times (2 d P_S)}$


In [141]:
### user inputs 
n,d = 100,5

# X - data matrix in n x d
X = np.random.randn(n,d)

# y are targets
y = np.random.randn(n,1)

r = np.linalg.matrix_rank(X)

# add bias term 
X = np.hstack([X, np.ones((n,1))])
d += 1

# number samples of D matrices in convex network
P_S = 10

# number of hidden layers in network 
m = 3

assert m <= P_S

# parameters (TODO: parameterize)
rho = 1e-5
step = 1e-5
beta = 1e-5

In [102]:
### TODO: figure out better way to sample D matrices

h = np.random.randn(d, P_S)

# TODO: assert that no duplicate columns
# n x P_S matrix, where each column is the diagonal entries for one D matrix
d_diags = X @ h >= 0

In [148]:

# F here is n x (2d*P_S) like paper
F = np.hstack([np.hstack([np.diag(d_diags[:,i].astype('uint8')) @ X for i in range(P_S)]),
    np.hstack([-np.diag(d_diags[:,i].astype('uint8')) @ X for i in range(P_S)[::-1]])])

# G is block diagonal 2*n*P_S x 2*d*P_s
Glist = [(2 * np.diag(d_diags[:,i].astype('uint8')) - np.eye(n)) @ X for i in range(P_S)]
G = block_diag(*Glist * 2)

# weights
# u contains u1 ... uP, z1... zP in one long vector
u = np.zeros((2 * d * P_S, 1))
# v contrains v1 ... vP, w1 ... wP in one long vector
v = np.zeros((2 * d * P_S, 1))

# slacks s1 ... sP, t1 ... tP
s = np.zeros((2 * P_S * n, 1))
for i in range(2 * P_S):
    # s_i = G_i v_i
    s[i*n:(i+1)*n] = Glist[i % P_S] @ v[i*d:(i+1)*d]

# dual variables
# lam contains lam11 lam12 ... lam1P lam21 lam22 ... lam2P
lam = np.zeros((2 * d * P_S,1))
# nu contains nu11 nu12 ... nu1P nu21 nu22 ... nu2P
nu = np.zeros((2 * n * P_S,1))

# precomputes: A in 2dP_s x 2dP_s
A = np.eye(2*d*P_S) + F.T @ F / rho + G.T @ G
# cholesky factorization
L = LA.cholesky(A)
# extra precompute for u update step
b_1 = F.T @ y / rho


In [105]:

# updates 

# first, conduct the primal update on u (u1...uP, z1...zP)
b = b_1 + v - lam + G.T @ (s - nu)
bhat = solve_triangular(L, b, lower=True)
u = solve_triangular(L.T, bhat, lower=False)

# second, perform updates of v and s
# TODO: parallelize v and s updates

# upates on v, (v1...vP, w1...wP)
for i in range(2 * P_S):
    inds = np.arange(d*i, d*(i+1))
    v[inds] = np.maximum(1 - beta/(rho * LA.norm(u[inds] + lam[inds])), 0) * (u[inds] + lam[inds])

# updates on s
for i in range(P_S):
    # update s_i
    s[i*n:(i+1)*n] = np.maximum(Glist[i] @ u[i*d:(i+1)*d] + nu[i*n:(i+1)*n], 0)
    # update t_i
    s[(i+P_S)*n:(i+P_S+1)*n] = np.maximum(Glist[i] @ u[(i+P_S)*d:(i+P_S+1)*d] + nu[(i+P_S)*n:(i+P_S+1)*n], 0)

# dual updates last
lam += lam + step / rho * (u - v)
nu += nu + step / rho * (G @ u - s)

In [146]:
# recover u1.... u_ms and alpha1 ... alpha_ms
u_star = np.zeros((m, d))
alpha_star = np.zeros((m,1))

v_star = v.reshape((P_S*2, d))[:P_S]
w_star = v.reshape((P_S*2, d))[P_S:P_S*2]

# critical number of neurons 
mstar = np.sum(~ np.isclose(LA.norm(v_star, axis=1), 0)) + np.sum(~ np.isclose(LA.norm(w_star, axis=1), 0))

if m > mstar:
    print("Network guranteed not optimal.")

j = 0
for i in range(P_S):
    if not np.isclose(LA.norm(v_star[i]), 0):
        u_star[j] = v_star[i] / np.sqrt(LA.norm(v_star[i]))
        alpha_star[j] = np.sqrt(LA.norm(v_star[i]))
        j += 1
    if j == m: break
for i in range(P_S):
    if not np.isclose(LA.norm(w_star[i]), 0):
        u_star[j] = w_star[i] / np.sqrt(LA.norm(w_star[i]))
        alpha_star[j] = - np.sqrt(LA.norm(w_star[i]))
        j += 1
    if j == m: break

print(f"Network has {j} nonzero neurons.")

Network guranteed not optimal.
Network has 0 nonzero neurons.
